# 🧬 Módulo 4: Modelado de Proteínas y Docking Molecular
## Actividad 4.3: Predicción de Estructuras con AlphaFold y ESMFold

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_04_modelado_proteinas_docking/03_alphafold_esmfold.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender los fundamentos de AlphaFold2 y ESMFold
- Utilizar ColabFold para predicciones rápidas
- Interpretar métricas de confianza (pLDDT, PAE)
- Usar la API de ESMFold para predicciones
- Comparar resultados de diferentes métodos de IA
- Identificar dominios y regiones desordenadas
- Seleccionar el método apropiado según el caso de uso

---

## 1. AlphaFold2: Fundamentos

### Arquitectura
- **Evoformer**: Procesa alineamientos evolutivos
- **Structure Module**: Genera coordenadas 3D
- **Recycling**: Itera para refinar la predicción

### Métricas de Confianza
- **pLDDT**: Confianza por residuo (0-100)
  - >90: Muy alta
  - 70-90: Alta
  - 50-70: Baja
  - <50: Muy baja
- **PAE**: Error de alineamiento predicho (matriz de confianza relativa)

## 2. Uso de ColabFold

ColabFold es la forma más práctica de usar AlphaFold.

### Opción A: ColabFold en Google Colab (Recomendado)

1. Visita: https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb
2. Pega tu secuencia
3. Ejecuta las celdas
4. Descarga los resultados

In [ ]:
# Instalación local de ColabFold (opcional)
# !pip install colabfold[alphafold]

# Para uso local completo, se requiere:
# - GPU con suficiente memoria (>8GB)
# - Bases de datos descargadas (~2.2 TB para uso completo)
# - O usar versión sin MSA (~mucho más rápido, menos preciso)

## 3. Análisis de Resultados de AlphaFold

In [ ]:
!pip install biopython py3Dmol matplotlib numpy

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from Bio import PDB
import py3Dmol

def visualizar_plddt(pdb_file):
    """
    Visualiza la proteína coloreada por pLDDT
    """
    with open(pdb_file) as f:
        pdb_data = f.read()
    
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'cartoon': {'colorscheme': 'pLDDT'}})
    view.zoomTo()
    return view

def analizar_pae(pae_json_file):
    """
    Analiza y visualiza la matriz PAE
    """
    with open(pae_json_file) as f:
        data = json.load(f)
    
    pae = np.array(data['pae'])
    
    plt.figure(figsize=(10, 8))
    plt.imshow(pae, cmap='viridis_r', vmin=0, vmax=30)
    plt.colorbar(label='PAE (Å)')
    plt.xlabel('Residuo')
    plt.ylabel('Residuo')
    plt.title('Predicted Aligned Error (PAE)')
    plt.show()
    
    print(f"PAE promedio: {pae.mean():.2f} Å")
    print(f"PAE mínimo: {pae.min():.2f} Å")
    print(f"PAE máximo: {pae.max():.2f} Å")

# Ejemplo de uso
# view = visualizar_plddt('resultado_alphafold.pdb')
# view.show()
# analizar_pae('resultado_pae.json')

## 4. ESMFold: Alternativa Rápida

ESMFold es significativamente más rápido que AlphaFold (sin necesidad de MSA).

### Ventajas:
- Muy rápido (~1 segundo por proteína)
- No requiere búsqueda evolutiva
- Fácil de ejecutar

### Desventajas:
- Ligeramente menos preciso que AlphaFold
- No genera PAE

## 5. Uso de ESMFold API

In [ ]:
import requests

def predecir_con_esmfold(sequence):
    """
    Predice estructura usando ESMFold API
    """
    url = "https://api.esmatlas.com/foldSequence/v1/pdb/"
    
    response = requests.post(url, data=sequence)
    
    if response.status_code == 200:
        # Guardar estructura
        with open('esmfold_prediction.pdb', 'w') as f:
            f.write(response.text)
        print("✓ Predicción completada con ESMFold")
        return 'esmfold_prediction.pdb'
    else:
        print(f"✗ Error: {response.status_code}")
        return None

# Ejemplo (limitado a secuencias cortas)
# secuencia_ejemplo = "MKTAYIAKQRQISFVKSHFSRQLEER"
# pdb_file = predecir_con_esmfold(secuencia_ejemplo)

## 6. Comparación AlphaFold vs ESMFold

In [ ]:
def comparar_predicciones(alphafold_pdb, esmfold_pdb):
    """
    Compara dos predicciones mediante RMSD
    """
    parser = PDB.PDBParser(QUIET=True)
    
    struct1 = parser.get_structure('af', alphafold_pdb)
    struct2 = parser.get_structure('esm', esmfold_pdb)
    
    atoms1 = []
    atoms2 = []
    
    for model in struct1:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    atoms1.append(residue['CA'])
    
    for model in struct2:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    atoms2.append(residue['CA'])
    
    super_imposer = PDB.Superimposer()
    min_len = min(len(atoms1), len(atoms2))
    super_imposer.set_atoms(atoms1[:min_len], atoms2[:min_len])
    
    rmsd = super_imposer.rms
    
    print(f"RMSD entre AlphaFold y ESMFold: {rmsd:.3f} Å")
    
    if rmsd < 2.0:
        print("→ Predicciones muy similares")
    elif rmsd < 5.0:
        print("→ Predicciones similares")
    else:
        print("→ Predicciones divergentes (revisar regiones de baja confianza)")
    
    return rmsd

# Ejemplo
# rmsd = comparar_predicciones('alphafold.pdb', 'esmfold.pdb')

## 7. Casos de Uso

### Cuándo usar AlphaFold:
- Proteínas con homólogos conocidos
- Complejos proteína-proteína
- Máxima precisión requerida
- Dominios múltiples

### Cuándo usar ESMFold:
- Cribado rápido de muchas secuencias
- Proteínas pequeñas (<400 residuos)
- Primeras aproximaciones
- Recursos computacionales limitados

## 8. Identificación de Dominios y Regiones Desordenadas

In [ ]:
def identificar_regiones_desordenadas(pdb_file, threshold=50):
    """
    Identifica regiones desordenadas basándose en pLDDT
    """
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    
    regiones_desordenadas = []
    region_actual = None
    
    for model in structure:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    plddt = residue['CA'].bfactor
                    res_id = residue.id[1]
                    
                    if plddt < threshold:
                        if region_actual is None:
                            region_actual = [res_id, res_id]
                        else:
                            region_actual[1] = res_id
                    else:
                        if region_actual:
                            regiones_desordenadas.append(tuple(region_actual))
                            region_actual = None
    
    if region_actual:
        regiones_desordenadas.append(tuple(region_actual))
    
    print(f"Regiones potencialmente desordenadas (pLDDT < {threshold}):")
    for inicio, fin in regiones_desordenadas:
        print(f"  Residuos {inicio}-{fin} ({fin-inicio+1} residuos)")
    
    return regiones_desordenadas

# Ejemplo
# regiones = identificar_regiones_desordenadas('alphafold.pdb')

## 9. Ejercicios Prácticos

### Ejercicio 1
Usa ColabFold para predecir la estructura de una proteína de tu interés. Analiza los valores de pLDDT y PAE.

### Ejercicio 2
Compara la predicción de AlphaFold con una estructura experimental del PDB.

### Ejercicio 3
Predice la misma proteína con ESMFold y compara los resultados con AlphaFold.

In [ ]:
# Espacio para ejercicios

## Referencias

1. Jumper et al. (2021). Highly accurate protein structure prediction with AlphaFold. *Nature* 596, 583–589
2. Lin et al. (2023). Evolutionary-scale prediction of atomic-level protein structure with a language model. *Science* 379, 1123-1130
3. Mirdita et al. (2022). ColabFold: making protein folding accessible to all. *Nature Methods* 19, 679–682

---

## 📚 Recursos Adicionales

### Herramientas y Servidores
- [ColabFold](https://github.com/sokrypton/ColabFold)
- [AlphaFold Database](https://alphafold.ebi.ac.uk/)
- [ESMFold API](https://esmatlas.com/about)

### Documentación
- [AlphaFold Paper (Nature 2021)](https://www.nature.com/articles/s41586-021-03819-2)
- [ESMFold Paper (Science 2023)](https://www.science.org/doi/10.1126/science.ade2574)

---

## ✅ Verificación de Aprendizaje

Al finalizar esta actividad deberías ser capaz de:

- ✅ Predecir estructuras con AlphaFold/ColabFold
- ✅ Usar ESMFold API para predicciones rápidas
- ✅ Interpretar valores de pLDDT y PAE
- ✅ Comparar predicciones de diferentes métodos
- ✅ Identificar regiones de baja confianza

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 4.3: Predicción con AlphaFold y ESMFold**

[![Anterior](https://img.shields.io/badge/⬅️_Actividad_4.2-Modelado_por_Homología-blue.svg)](02_modelado_homologia.ipynb)
[![Siguiente](https://img.shields.io/badge/Actividad_4.4_➡️-Preparación_de_Proteínas-green.svg)](04_preparacion_proteinas.ipynb)

---

📚 **[Volver al Módulo 4](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>